In [2]:
from datasets import load_dataset

ds = load_dataset("RZ412/PokerBench")

README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

(…)lop_500k_train_set_prompt_and_label.json:   0%|          | 0.00/607M [00:00<?, ?B/s]

(…)flop_60k_train_set_prompt_and_label.json:   0%|          | 0.00/62.0M [00:00<?, ?B/s]

(…)tflop_10k_test_set_prompt_and_label.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

(…)reflop_1k_test_set_prompt_and_label.json:   0%|          | 0.00/965k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/563200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11000 [00:00<?, ? examples/s]

In [3]:
ds.shape

{'train': (563200, 2), 'test': (11000, 2)}

In [12]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")

In [32]:
test_data = ds['test']

In [90]:
import requests
def query_llm(prompt, model):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": 0.1,
        "top_p": 0.95
    }
    response = requests.post("https://openrouter.ai/api/v1/completions", headers=headers, json=payload)
    response_data = response.json()
    if "choices" not in response_data:
        print(response_data)
        return
    txt = response.json()["choices"][0]["text"].strip()
    print(txt)
    return txt

In [96]:
model_deepseek = "deepseek/deepseek-r1:free"
model_llama = "meta-llama/llama-3.3-70b-instruct"
model_gemini = "google/gemini-2.0-flash-lite-preview-02-05:free"

In [97]:
def evaluate_model(predictions, ground_truths):
    action_correct = 0
    exact_match_correct = 0
    total = len(ground_truths)

    for pred, true in zip(predictions, ground_truths):
        pred_action = pred.lower().split()[0]
        true_action = true.lower().split()[0]
        
        # Action Accuracy: Does the action match?
        if pred_action == true_action:
            action_correct += 1
        
        # Exact Match Accuracy: Does the entire response match?
        if pred.lower() == true.lower():
            exact_match_correct += 1

    action_accuracy = (action_correct / total) * 100
    exact_match_accuracy = (exact_match_correct / total) * 100

    return action_accuracy, exact_match_accuracy


In [98]:
instructions = test_data['instruction']
outputs = test_data['output']

In [99]:
query_llm(instructions[0], model_gemini)

Check


'Check'

In [101]:
instructions = test_data['instruction']
outputs = test_data['output']

predictions = [query_llm(instruction, model_gemini) for instruction in instructions[:18]]

aa, em = evaluate_model(predictions, outputs[:18])

print(f"Action Accuracy (AA): {aa:.2f}%")
print(f"Exact Match Accuracy (EM): {em:.2f}%")

Check
Check
Check
Bet 10 chips
Check
Call
Call
Check
Call
Call
Call
Bet 20 chips
Check
Check
Check
Fold
Call
Call
Action Accuracy (AA): 83.33%
Exact Match Accuracy (EM): 77.78%
